# Calculate Sentiment score

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import os

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7efb82777310>)

In [5]:
min1 = True
min3 = " "


train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [6]:
train_data.drop_duplicates(subset=['document'], inplace=True)
test_data.drop_duplicates(subset=['document'], inplace=True)

In [7]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'] = train_data['document'].str.replace('^ +', "")
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

/tmp/ipykernel_28410/2535633430.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
/tmp/ipykernel_28410/2535633430.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "")
/tmp/ipykernel_28410/2535633430.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
/tmp/ipykernel_28410/2535633430.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경


In [15]:
# tokenizing
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)
tokenizer = Tokenizer(10000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

100%|██████████| 48852/48852 [06:18<00:00, 129.10it/s]


In [16]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]  # remove empty sample
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

max_len = 30
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [18]:
# sentiment analysis model -> using LSTM

from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
with tf.device('/cpu:0'):
    embedding_dim = 100
    hidden_units = 128
    vocab_size = 10000
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(hidden_units))
    model.add(Dense(1, activation='sigmoid'))

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15


2023-12-09 11:03:53.994034: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ef9a8063550 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-09 11:03:53.994072: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-12-09 11:03:54.809576: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


   2/1813 [..............................] - ETA: 3:37 - loss: 0.6944 - acc: 0.5156   

2023-12-09 11:03:56.339125: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1813/1813 [==============================] - ETA: 0s - loss: 0.4096 - acc: 0.8114
Epoch 1: val_acc improved from -inf to 0.82911, saving model to best_model.h5
1813/1813 [==============================] - 90s 47ms/step - loss: 0.4096 - acc: 0.8114 - val_loss: 0.3838 - val_acc: 0.8291
Epoch 2/15
   3/1813 [..............................] - ETA: 1:00 - loss: 0.3568 - acc: 0.8490

/root/anaconda3/envs/lim/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1813/1813 [==============================] - ETA: 0s - loss: 0.3487 - acc: 0.8478
Epoch 2: val_acc improved from 0.82911 to 0.84297, saving model to best_model.h5
1813/1813 [==============================] - 70s 38ms/step - loss: 0.3487 - acc: 0.8478 - val_loss: 0.3559 - val_acc: 0.8430
Epoch 3/15
1813/1813 [==============================] - ETA: 0s - loss: 0.3243 - acc: 0.8604
Epoch 3: val_acc improved from 0.84297 to 0.85156, saving model to best_model.h5
1813/1813 [==============================] - 67s 37ms/step - loss: 0.3243 - acc: 0.8604 - val_loss: 0.3445 - val_acc: 0.8516
Epoch 4/15
1813/1813 [==============================] - ETA: 0s - loss: 0.3039 - acc: 0.8717
Epoch 4: val_acc improved from 0.85156 to 0.85415, saving model to best_model.h5
1813/1813 [==============================] - 67s 37ms/step - loss: 0.3039 - acc: 0.8717 - val_loss: 0.3351 - val_acc: 0.8541
Epoch 5/15
1813/1813 [==============================] - ETA: 0s - loss: 0.2878 - acc: 0.8805
Epoch 5: val_acc impr

In [19]:
loaded_model = load_model('best_model.h5')

In [21]:
with tf.device('/cpu:0'):
    print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1527/1527 [==============================] - 18s 11ms/step - loss: 0.3425 - acc: 0.8514

 테스트 정확도: 0.8514


In [37]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new,verbose=0)) # 예측
    return score

In [38]:
# 각 리뷰들에 sentiment score 넣기
import os
directory = './raw_data/'
folder_list = os.listdir(directory)

In [42]:
import json
with tf.device('/cpu:0'): # colab에서 할 때는 이거 없애셈
    for folder in tqdm(folder_list):
        file_list = os.listdir(directory+"/"+folder)
        num = len(file_list)
        for i in range(0, num):
            file = directory+"/"+folder+"/%d.json" % i
            try:
                with open(file, encoding='UTF-8') as json_file:
                    json_data = json.load(json_file)
                    num = len(json_data["data"])
                    for i in range(0, num):
                        sc = sentiment_predict(json_data["data"][i]["review"])
                        json_data["data"][i]["sentiment score"] = sc

                    with open(file, 'w', encoding='UTF-8') as f:
                        json.dump(json_data, f, ensure_ascii=False, indent=3)
            except Exception:
                print(i, ".json", " error")

  0%|          | 0/6 [12:38<?, ?it/s]


KeyboardInterrupt: 

# Correlation score : spearman coefficient between rating and sentiment score

In [43]:
import os
import pandas as pd
import numpy
import json
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from pandas import json_normalize

from numpy.random import randn
from numpy.random import seed
from scipy.stats import spearmanr

In [ ]:
cname = ["rating", "sentiment", "correlation","review","spearman","sympathy","unsympathy"]

In [ ]:
movie_dit = {'1987':'1987',

'a taxi driver':'택시운전사',

'assassination':'암살',

'confidential assignment':'공조',

'intimate stranger':'완벽한 타인',

'the outlaws':'범죄도시'
}

In [ ]:

for folder in tqdm(folder_list):
    data_list = []
    file_list = os.listdir(directory+"/"+folder)
    num = len(file_list)
    for i in range(0, num):
        file = directory+"/"+folder+"/%d.json" % i
        try:
            with open(file, encoding='UTF-8') as json_file:
                json_data = json.load(json_file)
                num = len(json_data["data"])
                if(num>=3): # 리뷰 수가 3개 이상인 유저들의 데이터만
                    dt = []
                    ct = 0
                    t_1 = [] # rating, predict
                    t_2 = [] # sentiment score, test
                    t_3 = []
                    for i in range(0, num):
                        if(json_data["data"][x]["title"]==movie_dit[str(folder)]):
                            if(ct == 0):
                                if(len(json_data["data"][x]["review"])<=140):
                                    dt.append(int(json_data["data"][x]["rating"])) # 현재 영화에 대한 평점
                                    dt.append(json_data["data"][x]["sentiment score"]) # 현재 영화에 대한 감성점수
                                    dt.append(abs(int(json_data["data"][x]["rating"])/10 - json_data["data"][x]["sentiment score"])) # 평점과 감성점수의 편차
                                    dt.append(json_data["data"][x]["review"]) # 현재 영화에 대한 리뷰텍스트
                                    ct += 1
                        t_1.append(int(json_data["data"][x]["rating"]))
                        t_2.append(json_data["data"][x]["sentiment score"])
                    spearman, _ = spearmanr(t_1, t_2)
                    dt.append(spearman)
                    dt.append(json_data["sympathy"])
                    dt.append(json_data["unsympathy"])
                    data_list.append(dt)

        except Exception:
            print(i, ".json", " error")
        df = pd.DataFrame(data_list, columns=cname)
        fname = "./preprocessed_data/"+folder+".csv"
        df = df.dropna(axis=0)
        df.to_csv(fname,encoding="utf-8-sig")

# Labeling - 상위 30%는 1, 하위 30%는 0 -> 모델이 잘 학습될 수 있도록 train data에 이러한 제한을 둚

# outlaws가 평론을 생성할 영화임. 나머지 영화들로 모델 훈련

In [6]:
import os
import pandas as pd
file_list = [i for i in os.listdir("./preprocessed_data") if i.endswith(".csv") ] 

In [7]:
file_list

['1987.csv',
 'the outlaws.csv',
 'a taxi driver.csv',
 'assassination.csv',
 'intimate stranger.csv',
 'confidential assignment.csv']

In [8]:
import math
df_list = []
for file in file_list:
    if file != 'the outlaws.csv':
        df = pd.read_csv("./preprocessed_data/"+file, sep=",")
        df = df[["rating", "sentiment", "correlation","review","spearman"]]
        df["spearman"] = df["spearman"].abs() # 절대값으로 변환, 상관계수가 음의 방향으로 커도 상관관계의 정도가 큰 것이기 때문
        df = df.sort_values(by=['spearman'], axis=0, ascending=False) 
        df = df.reset_index(drop=True)
        total_num = len(df)
        trusted_num = math.floor(total_num * 0.3)
        distrusted_num = math.floor(total_num * 0.3)
        df["label"] = -1
        for i in range(trusted_num):
            df.loc[i,"label"] = 1

        for i in df.iloc[-distrusted_num:,:].index:
            df.loc[i,"label"] = 0
        df_list.append(df[df["label"]!=-1])

In [9]:
result_df = pd.concat(df_list, ignore_index=True)
result_df.head()

,rating,sentiment,correlation,review,spearman,label
0,10,0.233982,0.766018,이거 안본 사람이랑 말 안함,1.0,1
1,10,0.994989,0.005011,출연하신 모든 배우님들 정말 감사합니다. 김태리씨가 라디오를 녹음하시던 그 빨간 카...,1.0,1
2,8,0.658633,0.141367,드라마틱하게 기억에 남습니다.,1.0,1
3,10,0.965433,0.034567,아이에게 해줄말이 많았던영화,1.0,1
4,10,0.586564,0.413436,최고다. 그 시절을 기억하는 이들에게도 아닌 이들에게도.,1.0,1


In [10]:
result_df[["rating","sentiment","correlation"]].values

array([[1.00000000e+01, 2.33981848e-01, 7.66018152e-01],
       [1.00000000e+01, 9.94988620e-01, 5.01138000e-03],
       [8.00000000e+00, 6.58632815e-01, 1.41367185e-01],
       ...,
       [8.00000000e+00, 9.96811390e-01, 1.96811390e-01],
       [8.00000000e+00, 7.59411395e-01, 4.05886050e-02],
       [1.00000000e+01, 9.81144547e-01, 1.88554530e-02]])

# Training Model

In [11]:
import codecs
import numpy as np
from tqdm import tqdm
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split

numerical_features = result_df[["rating","sentiment","correlation"]].values.reshape(-1,3,1)
text_features = result_df["review"]
text_features = np.array([np.array(t) for t in text_features])
y = result_df["label"].values.astype('int32').reshape((-1,1))

x_train, x_test,x_feature,x_test_feature, y_train, y_test = train_test_split(text_features,numerical_features, y, test_size=0.2, 
                                                 random_state=34, stratify=y,shuffle=True)
x_train, x_val,x_feature,x_val_feature, y_train, y_val = train_test_split(x_train,x_feature, y_train, test_size=0.1, 
                                                 random_state=134,stratify=y_train, shuffle=True)

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
bert_preprocess = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilkobert_cased_preprocess/1")
bert_encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilkobert_cased_L-3_H-768_A-12/1")

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='review')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])

feature_input = tf.keras.layers.Input(shape=(3,1),)
feature_output = tf.keras.layers.Dense(3,activation="relu")(feature_input)
feature_output = tf.keras.layers.Flatten()(feature_output)


concatenated = tf.keras.layers.concatenate([l, feature_output])
concat_reshape = tf.keras.layers.Reshape((1,777))(concatenated) # reshape 2d to 3d
concat_out = tf.keras.layers.LSTM(32, return_sequences=True)(concat_reshape)
concat_out = tf.keras.layers.LSTM(32, return_sequences=True)(concat_out)
concat_out = tf.keras.layers.LSTM(32)(concat_out)
concat_out = tf.keras.layers.Dense(1, activation='sigmoid')(concat_out)
#concat_out = tf.keras.layers.LSTM(1, activation='sigmoid')(concatenated)

model = tf.keras.models.Model([text_input, feature_input], concat_out)



mc = ModelCheckpoint('best_review_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['acc'])
model.fit([x_train,x_feature], y_train, epochs=30,batch_size=32, callbacks=[mc],validation_data=([x_val,x_val_feature], y_val))
# multi input 이면 꼭!!! validation도 []로 묶어주는 거 잊지 말기..
result1 = model.evaluate([x_test,x_test_feature], y_test) 

Epoch 1/30
903/903 [==============================] - ETA: 0s - loss: 0.6870 - accuracy: 0.5429 - precision: 0.5379 - recall: 0.6073WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 549s 601ms/step - loss: 0.6870 - accuracy: 0.5429 - precision: 0.5379 - recall: 0.6073 - val_loss: 0.6845 - val_accuracy: 0.5432 - val_precision: 0.5551 - val_recall: 0.4361
Epoch 2/30
903/903 [==============================] - ETA: 0s - loss: 0.6808 - accuracy: 0.5619 - precision: 0.5587 - recall: 0.5889WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 540s 599ms/step - loss: 0.6808 - accuracy: 0.5619 - precision: 0.5587 - recall: 0.5889 - val_loss: 0.6822 - val_accuracy: 0.5591 - val_precision: 0.5592 - val_recall: 0.5595
Epoch 3/30
903/903 [==============================] - ETA: 0s - loss: 0.6779 - accuracy: 0.5679 - precision: 0.5658 - recall: 0.5834WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 540s 598ms/step - loss: 0.6779 - accuracy: 0.5679 - precision: 0.5658 - recall: 0.5834 - val_loss: 0.6803 - val_accuracy: 0.5550 - val_precision: 0.5546 - val_recall: 0.5601
Epoch 4/30
903/903 [==============================] - ETA: 0s - loss: 0.6754 - accuracy: 0.5762 - precision: 0.5695 - recall: 0.6243WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 540s 598ms/step - loss: 0.6754 - accuracy: 0.5762 - precision: 0.5695 - recall: 0.6243 - val_loss: 0.6803 - val_accuracy: 0.5587 - val_precision: 0.5518 - val_recall: 0.6274
Epoch 5/30
903/903 [==============================] - ETA: 0s - loss: 0.6739 - accuracy: 0.5759 - precision: 0.5712 - recall: 0.6089WARNING:tensorflow:Can save best model only with val_acc available, skipping.


...

903/903 [==============================] - 540s 598ms/step - loss: 0.6026 - accuracy: 0.6484 - precision: 0.6298 - recall: 0.7203 - val_loss: 0.7517 - val_accuracy: 0.5597 - val_precision: 0.5464 - val_recall: 0.7047
Epoch 28/30
903/903 [==============================] - ETA: 0s - loss: 0.5965 - accuracy: 0.6525 - precision: 0.6319 - recall: 0.7308WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 539s 597ms/step - loss: 0.5965 - accuracy: 0.6525 - precision: 0.6319 - recall: 0.7308 - val_loss: 0.7517 - val_accuracy: 0.5541 - val_precision: 0.5460 - val_recall: 0.6436
Epoch 29/30
903/903 [==============================] - ETA: 0s - loss: 0.5935 - accuracy: 0.6567 - precision: 0.6355 - recall: 0.7351WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 540s 598ms/step - loss: 0.5935 - accuracy: 0.6567 - precision: 0.6355 - recall: 0.7351 - val_loss: 0.7667 - val_accuracy: 0.5491 - val_precision: 0.5382 - val_recall: 0.6941
Epoch 30/30
903/903 [==============================] - ETA: 0s - loss: 0.5917 - accuracy: 0.6579 - precision: 0.6370 - recall: 0.7340WARNING:tensorflow:Can save best model only with val_acc available, skipping.


903/903 [==============================] - 539s 597ms/step - loss: 0.5917 - accuracy: 0.6579 - precision: 0.6370 - recall: 0.7340 - val_loss: 0.7808 - val_accuracy: 0.5538 - val_precision: 0.5468 - val_recall: 0.6299


NameError: name 'y_test_feature' is not defined

In [76]:
def prediction_result(n):
    if n > 0.4: 
        result=1
    else: result = 0
    return result

In [18]:
result1 = model.evaluate([x_test,x_test_feature], y_test) 

251/251 [==============================] - 134s 535ms/step - loss: 0.7892 - accuracy: 0.5578 - precision: 0.5507 - recall: 0.6285


In [77]:
test_prediction = model.predict([x_test,x_test_feature]) 

251/251 [==============================] - 134s 535ms/step


In [78]:
y_pred = list(map(prediction_result, test_prediction))

from sklearn.metrics import f1_score, accuracy_score

#  threshikd = 0.4
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f"f1 score : {f1} \n accuracy : {accuracy}")

f1 score : 0.6325695581014731 
 accuracy : 0.5522313637496884


In [19]:
#model.save("review_classification")

INFO:tensorflow:Assets written to: review_classification/assets


INFO:tensorflow:Assets written to: review_classification/assets


In [21]:
outlaws = pd.read_csv("./preprocessed_data/the outlaws.csv", sep=",")

In [49]:
outlaws = outlaws[["rating", "sentiment", "correlation","review","spearman"]]
outlaws["spearman"] = outlaws["spearman"].abs() # 절대값으로 변환, 상관계수가 음의 방향으로 커도 상관관계의 정도가 큰 것이기 때문
outlaws = outlaws.sort_values(by=['spearman'], axis=0, ascending=False) 
outlaws = outlaws.reset_index(drop=True)
total_num = len(outlaws)
trusted_num = math.floor(total_num * 0.5)
distrusted_num = total_num - trusted_num
for i in range(trusted_num):
    outlaws.loc[i,"label"] = 1

for i in outlaws.iloc[-distrusted_num:,:].index:
    outlaws.loc[i,"label"] = 0


In [50]:
numerical_features = outlaws[["rating","sentiment","correlation"]].values.reshape(-1,3,1)
text_features = outlaws["review"]
text_features = np.array([np.array(t).astype('<U140') for t in text_features])


In [51]:
predictions = model.predict([text_features,numerical_features])

508/508 [==============================] - 272s 535ms/step


In [53]:
outlaws

,rating,sentiment,correlation,review,spearman,label
0,8,0.428564,0.371436,생각보다 스토리범위가 너무좁았다 영화내내 칼빵만,1.0,1.0
1,9,0.917745,0.017745,윤계상 연기 잘하네요!,1.0,1.0
2,9,0.742317,0.157683,아무생각없이 즐겁게 보기 괜찮은 영화인듯 ㅋㅋ줄거린 별거 없다 ㅋㅋ 마동성 아저씨 ...,1.0,1.0
3,10,0.750091,0.249909,꾸루루루루잼!!!ㅎㅎㅎ,1.0,1.0
4,1,0.096552,0.003448,지루해 윤계상 연기변신은 괜찬,1.0,1.0
...,...,...,...,...,...,...
16235,10,0.574636,0.425364,꼭보세요. 진짜재미남요 1초도 안지루함,0.0,0.0
16236,5,0.679526,0.179526,마동석 캐릭터는 말안해도 매력적이며 윤계상은 재발견수준!하지만 피칠갑을 덧씌우며 벌...,0.0,0.0
16237,9,0.941423,0.041423,생각보다 덜 잔인해서 더 재밌게본듯,0.0,0.0
16238,10,0.889599,0.110401,중간중간 재미있는 요소들이 튀어나와서 더 즐겁게 봤네요!ㅋㅋㅋ,0.0,0.0


In [52]:
predictions

array([[0.6855989 ],
       [0.62675214],
       [0.5195645 ],
       ...,
       [0.5169674 ],
       [0.59649056],
       [0.470961  ]], dtype=float32)

In [72]:
def prediction_result(n):
    if n > 0.4: 
        result=1
    else: result = 0
    return result

result2 = list(map(prediction_result, predictions))

In [73]:
from sklearn.metrics import f1_score, accuracy_score

# 데이터 스플릿으로 y_valid와 모델 예측으로 y_pred를 구한 후 실행
# 모델 검정이 없다면 y_true값으로 y_valid 대체
f1 = f1_score(outlaws["label"].values, result2)
accuracy = accuracy_score(outlaws["label"].values, result2)

In [60]:
# threshold 0.5
print(f1)
print(accuracy)

0.5648028251912889
0.5447044334975369


In [63]:
# threshold 0.6
print(f1)
print(accuracy)

0.36002347614655816
0.529987684729064


In [74]:
# threshold 0.4
print(f1)
print(accuracy)

0.617892030848329
0.5423645320197045


In [75]:
outlaws["prediction_score"] = predictions
outlaws["predicted_label"] = result2 

In [79]:
outlaws

,rating,sentiment,correlation,review,spearman,label,prediction_score,predicted_label
0,8,0.428564,0.371436,생각보다 스토리범위가 너무좁았다 영화내내 칼빵만,1.0,1.0,0.685599,1
1,9,0.917745,0.017745,윤계상 연기 잘하네요!,1.0,1.0,0.626752,1
2,9,0.742317,0.157683,아무생각없이 즐겁게 보기 괜찮은 영화인듯 ㅋㅋ줄거린 별거 없다 ㅋㅋ 마동성 아저씨 ...,1.0,1.0,0.519565,1
3,10,0.750091,0.249909,꾸루루루루잼!!!ㅎㅎㅎ,1.0,1.0,0.201239,0
4,1,0.096552,0.003448,지루해 윤계상 연기변신은 괜찬,1.0,1.0,0.870583,1
...,...,...,...,...,...,...,...,...
16235,10,0.574636,0.425364,꼭보세요. 진짜재미남요 1초도 안지루함,0.0,0.0,0.556065,1
16236,5,0.679526,0.179526,마동석 캐릭터는 말안해도 매력적이며 윤계상은 재발견수준!하지만 피칠갑을 덧씌우며 벌...,0.0,0.0,0.407270,1
16237,9,0.941423,0.041423,생각보다 덜 잔인해서 더 재밌게본듯,0.0,0.0,0.516967,1
16238,10,0.889599,0.110401,중간중간 재미있는 요소들이 튀어나와서 더 즐겁게 봤네요!ㅋㅋㅋ,0.0,0.0,0.596491,1


In [80]:
new_outlaws = outlaws.sort_values(by=['prediction_score'], axis=0, ascending=False) 

In [82]:
new_outlaws = new_outlaws.reset_index(drop=True)
new_outlaws

,rating,sentiment,correlation,review,spearman,label,prediction_score,predicted_label
0,2,0.253529,0.053529,징그러운거 잔인한거 아이보니 그럼 비추,0.202797,0.0,0.998938,1
1,1,0.233134,0.133134,개무서움 덜덜 진짜 개무서움 덜덜더르,1.000000,1.0,0.998797,1
2,1,0.022273,0.077727,극혐도시 폭동도시 쓰레기,0.343166,0.0,0.998783,1
3,1,0.012172,0.087828,쓰레기임 폭력성만 강조한 냉무영화,0.866025,1.0,0.998727,1
4,10,0.524306,0.475694,범죄도시>>>범죄와의 전쟁,0.398527,0.0,0.998500,1
...,...,...,...,...,...,...,...,...
16235,10,0.524306,0.475694,동석이형 존멋 개꿀잼이여,0.175718,0.0,0.000606,0
16236,8,0.262457,0.537543,"마동석에의한,위한,의 도시",0.604933,1.0,0.000599,0
16237,1,0.323128,0.223128,조폭에서 벗어나질 못 하는,0.690174,1.0,0.000594,0
16238,8,0.605670,0.194330,범죄의도시범죄의도시,0.752139,1.0,0.000588,0


In [83]:
new_outlaws.to_csv("./result.csv",index=False)